In [5]:
import sys
import logging
import argparse
import os
import time
from os import path
import subprocess
import shutil
import asyncio
from datetime import datetime
from remote_daemons import Daemon, Wallet, StorageService

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')



In [6]:
def wait_for(callback, timeout=10):
    expires = time.time() + timeout
    while True:
        try:
            if callback():
                return
        except:
            pass
        if time.time() >= expires:
            raise RuntimeError("task timeout expired")
        time.sleep(.25)

def vprint(*args, timestamp=True, **kwargs):
    verbose = True
    if verbose:
        if timestamp:
            print(datetime.now(), end=" ")
        print(*args, **kwargs)
class SNNetwork:
    def __init__(self, datadir, *, binpath='../../build/bin',ss_binpath=".", sns=[], nodes=[],phase = 0):
        self.datadir = datadir
        # if not os.path.exists(self.datadir):
        #     os.makedirs(self.datadir)
        self.binpath = binpath
        self.ss_binpath = ss_binpath


        vprint("Using '{}' for data files and logs".format(datadir))

        nodeopts ={"oxend":self.binpath+'/oxend',"p2p_port":1000,"rpc_port":1001,
                "zmq_port":1002,"qnet_port":1003,"ss_port":1004,"datadir":"oxen/datadir"} 
        # dict(oxend=, datadir=datadir)

        self.sns = [Daemon(service_node=True,listen_ip=ip, **nodeopts) for ip in sns]
        self.nodes = [Daemon(listen_ip=ip,**nodeopts) for ip in nodes]
        
        self.all_nodes = self.sns + self.nodes

        self.wallets = []
        for idx,name in enumerate(('Alice', 'Bob', 'Mike')):
            self.wallets.append(Wallet(
                node=self.nodes[idx % len(self.nodes)],
                name=name,
                listen_ip=nodes[idx % len(self.nodes)],
                rpc_port=2001,
                rpc_wallet=self.binpath+'/oxen-wallet-rpc',
                datadir=datadir))

        self.alice, self.bob, self.mike = self.wallets

        # OMQ_path = "tcp://{}:{}".format(self.sns[0].listen_ip,self.sns[0].zmq_port)
        self.ss = [StorageService(storage=self.ss_binpath+'/oxen-storage', datadir=datadir,oxend_ip=sn.remote_ip,
                                  listen_ip =sn.remote_ip,
                                oxen_rpc="ipc:///root/"+sn.path+"/devnet/oxend.sock"
                                ,rpc_port=1005,omq_port=1006,verbose=True) 
                for sn in self.sns]
        
        if phase > 0:
            for node in self.all_nodes:
                node.client = True
            for wallet in self.wallets:
                wallet.client = True
        if phase > 3:
            for ss in self.ss:
                ss.client = True
        if phase <= 0:
            # Interconnections
            for i in range(len(self.all_nodes)):
                for j in (2, 3, 5, 7, 11):
                    k = (i + j) % len(self.all_nodes)
                    if i != k:
                        self.all_nodes[i].add_peer(self.all_nodes[k])

            vprint("Starting new oxend service nodes with RPC on {} ports".format(self.sns[0].listen_ip), end="")
            for sn in self.sns:
                vprint(" {}".format(sn.rpc_port), end="", flush=True, timestamp=False)
                sn.start()
            vprint(timestamp=False)
            vprint("Starting new regular oxend nodes with RPC on {} ports".format(self.nodes[0].listen_ip), end="")
            for d in self.nodes:
                vprint(" {}".format(d.rpc_port), end="", flush=True, timestamp=False)
                d.start()
            vprint(timestamp=False)

            vprint("Waiting for all oxend's to get ready")
            for d in self.all_nodes:
                d.wait_for_json_rpc("get_info")

        
        if phase <= 1:
            vprint("Oxends are ready. Starting wallets")
            for w in self.wallets:
                vprint("Starting new RPC wallet {w.name} at {w.listen_ip}:{w.rpc_port}".format(w=w))
                w.start()
            for w in self.wallets:
                w.ready()
                w.refresh()
                vprint("Wallet {w.name} is ready: {a}".format(w=w, a=w.address()))

            for w in self.wallets:
                w.wait_for_json_rpc("refresh")

        # Mine some blocks; we need 100 per SN registration, and we can nearly 600 on fakenet before
        # it hits HF16 and kills mining rewards.  This lets us submit the first 5 SN registrations a
        # SN (at height 40, which is the earliest we can submit them without getting an occasional
        # spurious "Not enough outputs to use" error).
        # to unlock and the rest to have enough unlocked outputs for mixins), then more some more to
        # earn SN rewards.  We need 100 per SN registration, and each mined block gives us an input
        # of 18.9, which means each registration requires 6 inputs.  Thus we need a bare minimum of
        # 6(N-5) blocks, plus the 30 lock time on coinbase TXes = 6N more blocks (after the initial
        # 5 registrations).
        # return
        if phase <=2:
            self.mine(100)
            for w in self.wallets:
                w.wait_for_json_rpc("refresh")
            vprint("Submitting first round of service node registrations: ", end="", flush=True)
            for sn in self.sns[0:5]:
                self.mike.register_sn(sn)
                vprint(".", end="", flush=True, timestamp=False)
            vprint(timestamp=False)
            if len(self.sns) > 5:
                vprint("Going back to mining", flush=True)

                self.mine(6*len(self.sns))

                self.print_wallet_balances()

                vprint("Submitting more service node registrations: ", end="", flush=True)
                for sn in self.sns[5:-1]:
                    self.mike.register_sn(sn)
                    vprint(".", end="", flush=True, timestamp=False)
                vprint(timestamp=False)
                vprint("Done.")

            self.print_wallet_balances()

            vprint("Mining 40 blocks (registrations + blink quorum lag) and waiting for nodes to sync")
            self.sync_nodes(self.mine(40))

            self.print_wallet_balances()
        if phase <=3:
            vprint("Starting Storage Serices")


            for s in self.ss:
                vprint("Storage @ {}".format(s.rpc_port), flush=True, timestamp=False)
                s.start()

            vprint("Sending fake lokinet/ss pings")
            for sn in self.sns:
                sn.ping()
        
        if phase <=4:
            all_service_nodes_proofed = lambda sn: all(x['quorumnet_port'] > 0 for x in
                    sn.json_rpc("get_n_service_nodes", {"fields":{"quorumnet_port":True}}).json()['result']['service_node_states'])

            vprint("Waiting for proofs to propagate: ", end="", flush=True)
            for sn in self.sns:
                wait_for(lambda: all_service_nodes_proofed(sn), timeout=220)
                vprint(".", end="", flush=True, timestamp=False)
            vprint(timestamp=False)
            for sn in self.sns[-1:]:
                self.mike.register_sn(sn)
                vprint(".", end="", flush=True, timestamp=False)
            self.sync_nodes(self.mine(1))
            time.sleep(10)
            for sn in self.sns:
                sn.send_uptime_proof()
            vprint("Done.")
        if phase <=5:
            vprint("Local Devnet SN network setup complete!")
            vprint("Communicate with daemon on ip: {} port: {}".format(self.sns[0].listen_ip,self.sns[0].rpc_port))
            configfile='config.py'
            with open(configfile, 'w') as filetowrite:
                filetowrite.write('#!/usr/bin/python3\n# -*- coding: utf-8 -*-\nlisten_ip=\"{}\"\nlisten_port=\"{}\"\nwallet_listen_ip=\"{}\"\nwallet_listen_port=\"{}\"\nwallet_address=\"{}\"\nexternal_address=\"{}\"'.format(self.sns[0].listen_ip,self.sns[0].rpc_port,self.mike.listen_ip,self.mike.rpc_port,self.mike.address(),self.bob.address()))




    def refresh_wallets(self, *, extra=[]):
        vprint("Refreshing wallets")
        for w in self.wallets + extra:
            w.refresh()
        vprint("All wallets refreshed")


    def mine(self, blocks=None, wallet=None, *, sync=False):
        """Mine some blocks to the given wallet (or self.mike if None) on the wallet's daemon.
        Returns the daemon's height after mining the blocks.  If blocks is omitted, mines enough to
        confirm regular transfers (i.e. 10 blocks).  If sync is specified, sync all nodes and then
        refresh all wallets after mining."""
        if wallet is None:
            wallet = self.mike
        if blocks is None:
            blocks = 10
        node = wallet.node
        vprint("Mining {} blocks to wallet {.name}".format(blocks, wallet))
        start_height = node.height()
        end_height = start_height + blocks
        node.mine_blocks(blocks, wallet)
        while node.rpc("/mining_status").json()["active"]:
            height = node.height()
            vprint("Mined {}/{}".format(height, end_height))
            time.sleep(0.05 if height >= end_height else 0.25)
        height = node.height()
        vprint("Mined {}/{}".format(height, end_height))

        if sync:
            self.sync_nodes(height)
            self.refresh_wallets()

        return height


    def sync_nodes(self, height=None, *, extra=[], timeout=10):
        """Waits for all nodes to reach the given height, typically invoked after mine()"""
        nodes = self.all_nodes + extra
        heights = [x.height() for x in nodes]
        if height is None:
            height = max(heights)
        if min(heights) >= height:
            vprint("All nodes already synced to height >= {}".format(height))
            return
        vprint("Waiting for all nodes to sync to height {}".format(height))
        last = None
        expiry = time.time() + timeout
        while nodes and time.time() < expiry:
            if heights[-1] < height:
                heights[-1] = nodes[-1].height()
            if heights[-1] >= height:
                heights.pop()
                nodes.pop()
                last = None
                continue
            if heights[-1] != last:
                vprint("waiting for {} [{} -> {}]".format(nodes[-1].name, heights[-1], height))
                last = heights[-1]
            time.sleep(0.1)
        if nodes:
            raise RuntimeError("Timed out waiting for node syncing")
        vprint("All nodes synced to height {}".format(height))


    def sync(self, extra_nodes=[], extra_wallets=[]):
        """Synchronizes everything: waits for all nodes to sync, then refreshes all wallets.  Can be
        given external wallets/nodes to sync."""
        self.sync_nodes(extra=extra_nodes)
        self.refresh_wallets(extra=extra_wallets)


    def print_wallet_balances(self):
        """Instructs the wallets to refresh and prints their balances (does nothing in non-verbose mode)"""
        vprint("Balances:")
        for w in self.wallets:
            b = w.balances(refresh=True)
            vprint("    {:5s}: {:.9f} (total) with {:.9f} (unlocked)".format(
                w.name, b[0] * 1e-9, b[1] * 1e-9))


    def __del__(self):
        vprint("shutting down all nodes and wallets") 
        for n in self.all_nodes:
            n.terminate()
        for w in self.wallets:
            w.terminate()


In [7]:
from ansible.parsing.dataloader import DataLoader
from ansible.inventory.manager import InventoryManager
from ssh_utils import connect
from concurrent.futures import ALL_COMPLETED, ThreadPoolExecutor, wait

def exec_on_host(server,cmd):
    logger.info(f"running {cmd} on {server}")
    client = connect(server)
    _, stdout, _ = client.exec_command(cmd)
    stdout.channel.recv_exit_status()
    client.close()
    
inventory_file_name = '../../ansible/inventory/hosts'
data_loader = DataLoader()
inventory = InventoryManager(loader = data_loader,
                            sources=[inventory_file_name])

nodes = inventory.get_groups_dict()['nodes']
service_nodes = inventory.get_groups_dict()['service_nodes']

In [4]:




logger.info("Clearing Data dirs")
servers = nodes + service_nodes
cmd = "pkill -9 oxen-wallet-rpc \n  pkill -9 oxend \n pkill -9 oxen-storage \n  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory"
executor = ThreadPoolExecutor(max_workers=10)

futures = [executor.submit(exec_on_host,server,cmd)  for server in servers]
done, not_done = wait(futures, return_when=ALL_COMPLETED)

if os.path.isdir("logs"):
            shutil.rmtree("logs/", ignore_errors=False, onerror=None)
            
logger.info("Waiting")
time.sleep(5)


02/03/2024 08:42:12 PM Clearing Data dirs
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 oxend 
 pkill -9 oxen-storage 
  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory on 49.13.199.194
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 oxend 
 pkill -9 oxen-storage 
  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory on 49.13.173.77
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 oxend 
 pkill -9 oxen-storage 
  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory on 23.88.98.217
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 oxend 
 pkill -9 oxen-storage 
  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory on 37.27.43.50
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 oxend 
 pkill -9 oxen-storage 
  rm -rf ~/oxen/datadir/* ~/oxen/datadirectory ~/datadirectory on 65.21.6.216
02/03/2024 08:42:12 PM running pkill -9 oxen-wallet-rpc 
  pkill -9 o

In [8]:
logger.info("starting new SNN")

print(service_nodes)
snn = SNNetwork(datadir="oxen/datadir",
                binpath="oxen/oxen-core/build/bin",
                ss_binpath="oxen/oxen-storage-server/build/httpserver",
                nodes=nodes,
                sns=service_nodes,
                phase=5
                )
        

02/03/2024 09:01:26 PM starting new SNN


['37.27.43.50', '65.21.6.216', '37.27.43.89', '37.27.43.85', '167.235.63.65', '5.75.171.41', '116.203.68.186', '195.201.147.111', '49.13.172.214', '128.140.66.163', '116.203.41.114', '116.203.49.0']
2024-02-03 21:01:26.957934 Using 'oxen/datadir' for data files and logs
oxen/oxen-storage-server/build/httpserver/oxen-storage 0.0.0.0 1005 --bind-ip=0.0.0.0 --data-dir=oxen/datadir/storage-37.27.43.50-1005 --oxend-rpc=ipc:///root/oxen/datadir/oxen-37.27.43.50-1001/devnet/oxend.sock --omq-port=1006
oxen/oxen-storage-server/build/httpserver/oxen-storage 0.0.0.0 1005 --bind-ip=0.0.0.0 --data-dir=oxen/datadir/storage-65.21.6.216-1005 --oxend-rpc=ipc:///root/oxen/datadir/oxen-65.21.6.216-1001/devnet/oxend.sock --omq-port=1006
oxen/oxen-storage-server/build/httpserver/oxen-storage 0.0.0.0 1005 --bind-ip=0.0.0.0 --data-dir=oxen/datadir/storage-37.27.43.89-1005 --oxend-rpc=ipc:///root/oxen/datadir/oxen-37.27.43.89-1001/devnet/oxend.sock --omq-port=1006
oxen/oxen-storage-server/build/httpserver/oxe

In [19]:
# snn.mike.node.json_rpc("sync_info").json()

#rpc("/mining_status").json()
# snn.mike.listen_ip="127.0.0.1"
# snn.mike.ready()
# snn.mike.node.remote_ip

Started RPC Wallet - Mike, on 127.0.0.1:2001


In [6]:
for d in snn.all_nodes:
    print(d.wait_for_json_rpc("get_info").json())

{'jsonrpc': '2.0', 'id': '0', 'result': {'alt_blocks_count': 0, 'block_size_limit': 600000, 'block_size_median': 300000, 'block_weight_limit': 600000, 'block_weight_median': 300000, 'cumulative_difficulty': 215, 'database_size': 700416, 'devnet': True, 'difficulty': 1, 'free_space': 32789483520, 'grey_peerlist_size': 5, 'height': 215, 'height_without_bootstrap': 215, 'immutable_block_hash': '', 'immutable_height': 0, 'incoming_connections_count': 5, 'last_lokinet_ping': 1706901963, 'last_storage_server_ping': 1706902295, 'mainnet': False, 'nettype': 'devnet', 'offline': False, 'ons_counts': [0, 0, 0], 'outgoing_connections_count': 5, 'pulse_ideal_timestamp': 1706915830, 'pulse_target_timestamp': 1706902304, 'service_node': True, 'start_time': 1706901699, 'status': 'OK', 'status_line': 'v9.2.0; Height: 215, SN: active, proof: 7m41s, storage: 5s, lokinet: 5m37s', 'target': 120, 'target_height': 101, 'testnet': False, 'top_block_hash': '9bb7ef134714c0cd64dceecffbeabdfae64b3f76c3e5db1d4177

In [14]:
snn.mike.node.mine_blocks(10, snn.mike)
# snn.mike.node.rpc('/start_mining', {
#             "miner_address": snn.mike.address(),
#             "threads_count": 1,
#             "num_blocks": 1,
#             "slow_mining": True
#         }).json()

{'status': 'Failed, wrong address'}

In [8]:
for n in snn.nodes:
    print(n.json_rpc("sync_info").json())
# for n in ss.all_nodes:
#     print(n.json_rpc("hard_fork_info").json()['result']['status'])
# for idx,n in enumerate(ss.all_nodes):
#     print(idx,n.json_rpc("get_service_node_status").json())

{'jsonrpc': '2.0', 'id': '0', 'result': {'height': 216, 'next_needed_pruning_seed': 0, 'overview': '[]', 'peers': [{'info': {'address': '167.235.63.65:1000', 'address_type': 1, 'avg_download': 0, 'avg_upload': 0, 'connection_id': '760e7f2c0301440cb19e4630807fb3ac', 'current_download': 0, 'current_upload': 0, 'height': 216, 'host': '167.235.63.65', 'incoming': False, 'ip': '167.235.63.65', 'live_time': 626, 'local_ip': False, 'localhost': False, 'peer_id': 'ca7792cc87b42089', 'port': '1000', 'pruning_seed': 0, 'recv_count': 216205, 'recv_idle_time': 25, 'rpc_port': 0, 'send_count': 212976, 'send_idle_time': 25, 'state': 'normal', 'support_flags': 1}}, {'info': {'address': '37.27.43.89:1000', 'address_type': 1, 'avg_download': 0, 'avg_upload': 0, 'connection_id': '9ce0f6f10aab446cafcf608e78353e4a', 'current_download': 0, 'current_upload': 0, 'height': 216, 'host': '37.27.43.89', 'incoming': False, 'ip': '37.27.43.89', 'live_time': 626, 'local_ip': False, 'localhost': False, 'peer_id': 'f

In [9]:
snn.nodes[0].json_rpc("get_all_service_nodes",
                     {"fields":{"quorumnet_port":True,
                                "registration_height":True,
                                "last_uptime_proof": True,
                                "swarm_id": True,
                                }}).json()

{'jsonrpc': '2.0',
 'id': '0',
 'result': {'service_node_states': [{'last_uptime_proof': 1706902066,
    'quorumnet_port': 1003,
    'registration_height': 213,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901900,
    'quorumnet_port': 1003,
    'registration_height': 173,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901900,
    'quorumnet_port': 1003,
    'registration_height': 173,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901900,
    'quorumnet_port': 1003,
    'registration_height': 173,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901900,
    'quorumnet_port': 1003,
    'registration_height': 173,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901839,
    'quorumnet_port': 1003,
    'registration_height': 101,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901900,
    'quorumnet_port': 1003,
    'registration_height': 173,
    'swarm_id': 0},
   {'last_uptime_proof': 1706901839,
    'quorumnet_port': 1003,
    'registration_height': 101,
    'swarm_id': 0},


In [10]:
import requests
def json_rpc_https(self, method, params=None, *, timeout=10):
    """Sends a json_rpc request to the rpc port.  Returns the response object."""
    json = {
            "jsonrpc": "2.0",
            "id": "0",
            "method": method,
            }
    json["params"] ={}
    if params:
        json["params"] = params

    return requests.post('https://{}:{}/storage_rpc/v1'.format(self.listen_ip, self.rpc_port), verify=False, json=json, timeout=timeout)

json_rpc_https(snn.ss[2],"get_snodes_for_pubkey",None).content
# storage_services[0].rpc_port

/home/pouya/projects/sarbazi/oxen-orchestrator/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '37.27.43.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


b"invalid request: Required field 'pubkey' missing"

In [20]:
for sn in ss.sns:
    sn.ping()